# A demo using Hydrogen Hamiltonian with GPT-QE.

In [15]:
import json, os, torch
from qwrapper.hamiltonian import compute_ground_state
from qwrapper.obs import PauliObservable
from gqe.mingpt.model import GPT
from gqe.mingpt.trainer import Trainer
from gqe.mingpt.utils import set_seed
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.mingpt.cost import EnergyCost
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor
import logging

logging.getLogger('benchmark.molecule').setLevel(logging.WARNING)

nqubit = 12
n_gates = 60
iter = 500
n_sample = 50
seed = 3047
ignore_cache = True
# distances = [1.0, 1.33, 1.5, 2.0, 2.5, 3.0]  # choices of the distance between two atoms
distances = [2.5, 3.0]
model_type = 'gpt2'
transformation = 'jordan-wigner'
is_bravyi = transformation == 'bravyi-kitaev'
MODEL_FILEBASE = '../saved_models/{}_model_beh2_sto3g_{}_{}_{}.json'
TRAJECTORY_FILEBASE = '../output/{}_trajectory_beh2_sto3g_{}_{}_{}.txt'
ENERGY_FILEBASE = '../output/{}_energy_beh2_sto3g_{}_{}_{}.txt'
OTHER_FILEBASE = '../output/{}_other_beh2_sto3g_{}_{}_{}.json'

# Training

In [16]:

def find_ground_state_energy(distance, seed, ignore_cache=False):
    geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {distance}\n" + f"H 0.0 0.0 {2 * distance}\n"
    molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
    set_seed(seed)
    # prepare file
    model_output = MODEL_FILEBASE.format(model_type, str(distance), transformation, seed)
    other_output = OTHER_FILEBASE.format(model_type, str(distance), transformation, seed)
    trajectory_output = TRAJECTORY_FILEBASE.format(model_output, str(distance), transformation, seed)

    if not ignore_cache and os.path.exists(model_output):
        return

    # prepare Hamiltonian
    hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=is_bravyi)

    ge = compute_ground_state(hamiltonian)
    print("ground state:", ge)

    # prepare operator_pool
    uccsd = UCCSD(nqubit, molecule)
    paulis = uccsd.paulis
    paulis.append(PauliObservable("IIIIIIIIIIII"))
    initializer = HFStateInitializer(n_electrons=4)
    print("hf state:", hamiltonian.exact_value(initializer.init_circuit(nqubit, [], "qulacs")))

    pool = DefaultOperatorPool(paulis)
    cost = EnergyCost(hamiltonian, initializer, pool,
                      [0.00625, -0.00625, 0.0125, -0.0125, 0.025, -0.025, 0.05, -0.05, 0.1, -0.1])

    model_config = GPT.get_default_config()
    model_config.model_type = model_type
    model_config.vocab_size = cost.vocab_size()
    model_config.block_size = cost.vocab_size()
    model_config.n_gates = n_gates  # The number of gates for each circuit
    model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
    model_config.embd_pdrop = 0.1
    model_config.resid_pdrop = 0.1
    model_config.attn_pdrop = 0.1
    model_config.energy_offset = 14
    model = GPT(model_config, cost)

    # create a Trainer object

    train_config = Trainer.get_default_config()
    train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
    train_config.max_iters = iter
    train_config.num_workers = 10
    train_config.n_samples = n_sample

    file_monitor = FileMonitor()
    trainer = Trainer(train_config, model)
    trainer.set_callback('on_batch_end',
                         callback=DefaultCallback(model, monitors=[PrintMonitor(), file_monitor]).generate())
    trainer.run()
    torch.save(model.state_dict(), model_output)
    file_monitor.save(trajectory_output)

    m = {"distance": distance,
         "exact_energy": ge,
         "computed_energy": file_monitor.min_energy,
         "computed_indices": file_monitor.min_indices,
         "n_gates": n_gates, "seed": seed}
    with open(other_output, "w") as f:
        f.write(json.dumps(m))


In [17]:
for d in distances:
    find_ground_state_energy(d, seed, ignore_cache=ignore_cache)

converged SCF energy = -15.1630689782417
ground state: -15.351491075144125
hf state: -15.163068978241704
number of parameters: 87.34M
running on device mps
iter_dt 0.00s; iter 0: train loss 0.16225 temperature: 5
mean_logits tensor([-14.8991, -14.9721, -14.8540, -15.0622, -14.9301, -14.9304, -14.8507,
        -14.9220, -14.9402, -14.9367, -14.9286, -15.0490, -14.8971, -14.9487,
        -14.9017, -14.9248, -14.9157, -15.0625, -14.9330, -14.9502, -14.8442,
        -14.8251, -14.9642, -14.9692, -14.9388, -14.9808, -15.0340, -14.9217,
        -14.9180, -14.9729, -14.9355, -14.8812, -15.0982, -14.9390, -14.9594,
        -14.9196, -15.0458, -15.0558, -14.9680, -15.0097, -14.7745, -14.9584,
        -14.9157, -14.9692, -14.8295, -14.9861, -14.9945, -14.9935, -14.9585,
        -14.9372], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-15.1045, -15.1296, -15.1380, -15.0506, -14.9719, -15.1413, -15.0511,
        -15.0613, -15.1355, -15.1068, -15.0996, -15.0912, -14.9453, -15.0409,
    


KeyboardInterrupt



# Figure

In [ ]:
min_d = distances[0] - 0.1
max_d = distances[len(distances) - 1] + 0.1
n_bin = 100

xs = []
ys = []
ys3 = []
initializer = HFStateInitializer(n_electrons=4)
for j in range(n_bin):
    d = min_d + (max_d - min_d) / (n_bin - 1) * j
    geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {d}\n" + f"H 0.0 0.0 {2 * d}\n"
    molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
    hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)
    ge = compute_ground_state(hamiltonian)
    scf = hamiltonian.exact_value(initializer.init_circuit(nqubit, [], "qulacs"))
    xs.append(d)
    ys.append(ge)
    ys3.append(scf)


In [ ]:
xs2 = []
ys2 = []

for d in distances:
    xs2.append(d)
    with open(OTHER_FILEBASE.format(model_type, d, transformation, seed)) as f:
        ys2.append(json.loads(f.readline())['computed_energy'])

import matplotlib.pyplot as p

# p.grid('-')
p.plot(xs, ys, label='exact', linewidth=1, color='blue')
p.plot(xs2, ys2, label='computed', marker='x', linewidth=0, color='green')
p.plot(xs, ys3, label='hf', linewidth=1, color='gray')
# p.xlim([0.9, 2.3])
# p.ylim([-15.6, -15.2])
p.xlabel('bond length (angstrom)')
p.ylabel('energy value (Hartree)')
p.title('GPT-QE with BeH2 Hamiltonian (sto-3g basis, 12-qubits, 40-gates)')
p.legend()
p.show()